# Monte Carlo Simulation (Backend Flow)

### Import the file & extract the ticker, shares, and purchase price

In [1]:
import pandas as pd
import numpy as np
from sympy.physics.units import days

df = pd.read_csv("data.csv")

print(df)

  Ticker  Shares  Purchase_Price
0   AAPL      50             180
1   GOOG      40             150


##### Additionally, calculate the portfolio value

In [2]:
tickers = df['Ticker'].to_numpy()
shares = df['Shares'].to_numpy()
purchase_price = df['Purchase_Price'].to_numpy()

portfolio_value = shares * purchase_price

print(tickers)
print(shares)
print(purchase_price)
print(portfolio_value.sum())

['AAPL' 'GOOG']
[50 40]
[180 150]
15000


### Now, we must return the historical data for the closings for our selected tickers

In [11]:
from typing import NamedTuple
from massive import RESTClient
import os
from dotenv import load_dotenv
import numpy as np
import statistics

load_dotenv()
api_key = os.getenv("MASSIVE_API_KEY")

client = RESTClient(api_key=api_key)

ticker_closes: dict[str, list[float]] = {}
ticker_daily_changes: dict[str, list[float]] = {}  # This is your returns
ticker_expected_returns: dict[str, float] = {}
ticker_volatility: dict[str, float] = {}

for ticker in tickers:
    ticker_closes[ticker] = []

    for a in client.list_aggs(
        ticker,
        1,
        "day",
        "2023-01-05",
        "2025-01-08",
        sort="asc",
    ):
        ticker_closes[ticker].append(a.close)

    # Calculate daily percentage changes
    ticker_daily_changes[ticker] = []
    closes = ticker_closes[ticker]

    for i in range(1, len(closes)):
        current_close = closes[i]
        previous_close = closes[i - 1]
        pct_change = ((current_close - previous_close) / previous_close) * 100
        ticker_daily_changes[ticker].append(pct_change)

    # Calculate expected return (geometric mean, annualized)
    changes = ticker_daily_changes[ticker]
    daily_returns = [(1 + c/100) for c in changes]
    geometric_mean = np.prod(daily_returns) ** (1/len(daily_returns))
    ticker_expected_returns[ticker] = ((geometric_mean ** 252) - 1) * 100

    # Calculate volatility (standard deviation, annualized)
    daily_std = statistics.stdev(changes)
    ticker_volatility[ticker] = daily_std * np.sqrt(252)

# Display results
print(f"Ticker closes (AAPL, first 5): {ticker_closes['AAPL'][:5]}")
print(f"Daily % changes (AAPL, first 5): {ticker_daily_changes['AAPL'][:5]}")
print(f"\nExpected Returns (Annualized):")
for ticker in tickers:
    print(f"  {ticker}: {ticker_expected_returns[ticker]:.2f}%")
print(f"\nVolatility/Risk (Annualized Std Dev):")
for ticker in tickers:
    print(f"  {ticker}: {ticker_volatility[ticker]:.2f}%")


def calculate_portfolio_variance(
    tickers: list[str],
    weights: list[float],
    ticker_returns: dict[str, list[float]]
) -> float:
    """
    Calculate portfolio variance given tickers, weights, and historical returns.

    Formula: σ²_p = Σᵢ Σⱼ wᵢ wⱼ Cov(rᵢ, rⱼ)

    Args:
        tickers: List of ticker symbols
        weights: List of portfolio weights (must sum to 1.0)
        ticker_returns: Dict of daily returns for each ticker

    Returns:
        Annualized portfolio variance (as percentage²)
    """
    import numpy as np

    # Validate weights sum to 1
    if not np.isclose(sum(weights), 1.0):
        raise ValueError(f"Weights must sum to 1.0, got {sum(weights)}")

    # Build returns matrix (each column = ticker's returns)
    returns_matrix = []
    for ticker in tickers:
        returns_matrix.append(ticker_returns[ticker])

    # Transpose so each row = day, each column = ticker
    returns_matrix = np.array(returns_matrix).T

    # Calculate covariance matrix (daily)
    cov_matrix = np.cov(returns_matrix, rowvar=False)

    # Calculate portfolio variance: w^T * Cov * w
    weights_array = np.array(weights)
    portfolio_variance_daily = weights_array.T @ cov_matrix @ weights_array

    # Annualize (multiply by 252 trading days)
    portfolio_variance_annual = portfolio_variance_daily * 252

    return portfolio_variance_annual


def calculate_portfolio_std(
    tickers: list[str],
    weights: list[float],
    ticker_returns: dict[str, list[float]]
) -> float:
    """
    Calculate portfolio standard deviation (volatility).

    Returns:
        Annualized portfolio std deviation (as percentage)
    """
    variance = calculate_portfolio_variance(tickers, weights, ticker_returns)
    return np.sqrt(variance)


# Usage Example
tickers_list = ['AAPL', 'GOOG']
portfolio_weights = [0.6, 0.4]  # 60% AAPL, 40% GOOG

# Pass ticker_daily_changes (not ticker_returns)
portfolio_var = calculate_portfolio_variance(tickers_list, portfolio_weights, ticker_daily_changes)
portfolio_std = calculate_portfolio_std(tickers_list, portfolio_weights, ticker_daily_changes)

print(f"\nPortfolio Analysis (60% AAPL, 40% GOOG):")
print(f"  Portfolio Variance: {portfolio_var:.2f}%²")
print(f"  Portfolio Std Dev (Risk): {portfolio_std:.2f}%")

# Compare to individual stocks (use ticker_volatility)
print(f"\nComparison:")
print(f"  AAPL Std Dev: {ticker_volatility['AAPL']:.2f}%")
print(f"  GOOG Std Dev: {ticker_volatility['GOOG']:.2f}%")
print(f"  Portfolio Std Dev: {portfolio_std:.2f}% (lower due to diversification!)")

Ticker closes (AAPL, first 5): [192.42, 191.73, 188.04, 184.4, 186.86]
Daily % changes (AAPL, first 5): [-0.35859058309946873, -1.9245814426537309, -1.9357583492873784, 1.3340563991323253, -0.540511612972289]

Expected Returns (Annualized):
  AAPL: 27.73%
  GOOG: 28.72%

Volatility/Risk (Annualized Std Dev):
  AAPL: 22.60%
  GOOG: 28.18%

Portfolio Analysis (60% AAPL, 40% GOOG):
  Portfolio Variance: 418.49%²
  Portfolio Std Dev (Risk): 20.46%

Comparison:
  AAPL Std Dev: 22.60%
  GOOG Std Dev: 28.18%
  Portfolio Std Dev: 20.46% (lower due to diversification!)


# Calculate Expected Returns

In [20]:
def expected_gain(days, portfolio_value, tickers, weights, ticker_expected_returns):
    """Super simple expected gain calculation"""

    # Portfolio annual return
    annual_return = sum(
        weights[i] * ticker_expected_returns[tickers[i]]
        for i in range(len(tickers))
    )

    # Scale to days and calculate gain
    return portfolio_value * (annual_return / 252 * days) / 100


# Usage
gain = expected_gain(20, 100000, ['AAPL', 'GOOG'], [0.6, 0.4], ticker_expected_returns)
print(f"Expected 10-day gain: ${gain:,.2f}")

Expected 10-day gain: $2,232.22


# Monte Carlo Visual